In [1]:
import shlex
import os
from subprocess import check_output
import sys
import torch
import numpy
import pyvista as pv

os.chdir(check_output(shlex.split("git rev-parse --show-toplevel")).strip().decode('ascii'))
from config.config_parser import read_config
sys.path.append("utils")
from helpers import *

In [2]:
import mesh_operations
from IPython.display import display
from notebooks.utils_ipywidgets import select_experiment

device = get_device()

2021-02-08 18:44:45,017 [INFO] No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


### Select experiment

In [3]:
w = select_experiment(); display(w)

Dropdown(description='Experiment:', index=499, options=('2020-10-12_03-44-42', '2020-10-12_03-07-22', '2020-10…

In [ ]:
import ExperimentClass

exp_dir = os.path.join("output", w.value)
experiment = ExperimentClass.ComaExperiment(exp_dir)
experiment.load_model()

#### Load everything as testing data

In [9]:
dataset = load_cardiac_dataset(experiment.config)
_, _, dataloader = get_loader(dataset=dataset, nTraining=0, nVal=0, batch_size=1, num_workers=4, shuffle=False)

2021-02-08 18:46:18,077 [INFO] Loading pre-aligned mesh data from data/transforms/cached/2ch_segmentation__LV__ED__scaled.pkl


In [10]:
experiment.model.eval()
torch.no_grad()
for i, (mesh, id) in enumerate(dataloader):
    if experiment.model.is_variational:
        mu, log_var = experiment.model.encoder(x=mesh)
        z = mu # coma.sampling(mu, log_var)
    else:
        z = experiment.model.encoder(x=mesh)
    mesh_r = experiment.model.decoder(z)

KeyboardInterrupt: 

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
def f(x, reconstructed=True, normalized=True):
  _mesh = mesh_r if reconstructed else mesh
  _mesh = _mesh[x].detach().numpy() if normalized else dataset.mean + dataset.std * _mesh[x].detach().numpy()

  plotter = pv.Plotter(notebook=True)
  plotter.add_mesh(_mesh)
  plotter.show(interactive=True)
  plotter.enable()

interact(f, 
  x=widgets.SelectionSlider(options=range(200)),
  #x=widgets.Dropdown(options=range(len(mesh)), description='mesh index:', disabled=False),
  reconstructed=True,
  normalized=True
)